In [101]:
from pymongo import MongoClient
import pandas as pd


In [102]:
import tools.mongo as mongo

In [103]:
df_companies=mongo.getCompanies(mongo.localConection())

In [104]:
#df_companies.head()


In [105]:
sum(df_companies.deadpooled_year.value_counts().values)

929

In [106]:
#df_companies[df_companies.deadpooled_year.isnull()]

## Api google places

In [211]:
import time
import requests
import dotenv
import os
dotenv.load_dotenv()

API_KEY = os.getenv("GOOGLE_KEY")

In [108]:
#url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [2]:
params={
    'key':API_KEY,
    'location':'-33.8670522,151.1957362',
    'radius':'1500',
    'type':'restaurant'
}

In [33]:

url='https://maps.googleapis.com/maps/api/place/findplacefromtext/json'


In [34]:
params={
    'key':API_KEY,
    'input':'Bar',
    'inputtype':'textquery',
    'fields':'name,rating',
    'locationbias':'circle:10000@47.6918452,-122.2226413'
}
res = requests.get(url, params=params)
res.json()

{'candidates': [{'name': 'Screwdriver Bar', 'rating': 4.6}], 'status': 'OK'}

In [233]:
def validNum(num):
    try:
        float(num)
        return True
    except:
        return False


def nearby(lat="",lon="",radius="",googleType="", name="",pagetoken=""):
    '''
    makes a call to nearbysearch of google api and return 20 places
    '''
    nearByURL="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params={}
    if pagetoken!="":
        params['pagetoken']=pagetoken
    elif validNum(lat) and validNum(lon) and validNum(radius):
        #Mandatory if not pagetoken
        params['location']='{},{}'.format(lat,lon)
        print(params['location'])
        params['radius']=str(radius)
        
        #Recomended
        if googleType!="":
            params['type']=googleType
        #Optional
        if name!='name':
            params['name']=name
        
    params['key']=API_KEY
    #print(params)
    if len(params)==0:
        raise ValueError  
    res=requests.get(nearByURL, params=params).json()
    #print(res)
    return res

def mapping(result):
    info={}
    if 'name' in result:
        info["name"]=result['name']
    if 'geometry' in result:
        info["geometry"]=result['geometry']['location']
    if 'rating' in result:
        info["rating"]=result['rating']
    #print(info)
    return info

def getNearbyplaces(lat="",lon="",radius="",googleType="", name=""):
    print("entro")
    res=nearby(lat,lon,radius,googleType,name)
    print('Recibido')
    places=[]
    results=res['results']
    for i in range(len(results)):
        places.append(mapping(results[i]))
    while ('next_page_token' in res):
        time.sleep(2)
        print("===============")
        print(res["next_page_token"])
        res=nearby(pagetoken=res["next_page_token"])
        results=res['results']
        print(res)
        #print("Siguente llamada",len(results))
        print('next_page_token' in res)
        for i in range(len(results)):
            places.append(mapping(results[i]))
        print(len(places))
    return places
    
    

   

In [234]:
a=getNearbyplaces(47.6918452,-122.2226413,10000)

entro
47.6918452,-122.2226413
Recibido
CqQCFQEAAA1U-W6lp-8omUeI0nXwQP1vxNwl3F6iIjuzy21s2bX8KZBPm06EMAiApL3-mP3QMxe9OE2Z8qqhLTfIGSRiNSjfc6fbuRB-QLdJHbeGic7gPPntTdfTMupCXM9NF8wBfHLMrYsYiIc_wUmmcmIbVaeNo8kf0BOEsD3ZlsarWuMfsEpbsk9pRjnhclBmwDCVWClT_K63vgJisWLl3G2P8BylbeEkQAyJyX1C8d_FE48tjLEyZ2LzmM1TdtJJtnM3OhJTn-Pi9occFhBd0o3ZwCgk6tl4uxX-MIg3qKp9k2jTRDeFQGa215lT3W0ptpoSkWtWND4Yu_J67YxR9ikPpZygiaPlXMPSl3aPWp73vez05cAVG3WJUQa-if2gmw_yHhIQYInpttLvei_F3bocf6N4WBoUxbqaslt8vDJJHTdARvG-2Cb11Ig
{'html_attributions': [], 'next_page_token': 'CsQDtQEAAN1lobBFwvh7D9EE-qlJqtsH4903q9_84M6KKdwoeSzNx6ooVwRZ7HLP9lZlYLvpR9OyhQ4LiqIl9bxrcJKqtzvN9eVG-3-MlOfCWwGyEEBGpOkpxX6-x7owRRQIUIwGo89EbwwLCRb-7ipjCH_f9hYXOwuPLaUg6bw3KSUkDdBsRO1xCcWLSrQa4jIQuiQV2Tfo6GyaDjsK4rHKuMLX93CQDtXItb5S2iT42BMC0YYD_KpSKuGdROvnj9wrBvl8Az4R0doDRwB8BTSR1_L3H0Gaox00WpzhFbnUKLs-a0wuFLX0CPo3bUCrcQRXWa6RybifqYf9fQVhJ9cHE_KDPesZEad5x-DocR9s6-NPJGLMQ4eaW0uBEW8XmGQllUcKUILdAbT_j7v4vQUNh7JHW2ih_e0RnH6l6u1dOESQz9YtpLv-Z22ijjmoBencE6hpZIuA0r8sHLH

CsQDtQEAAN1lobBFwvh7D9EE-qlJqtsH4903q9_84M6KKdwoeSzNx6ooVwRZ7HLP9lZlYLvpR9OyhQ4LiqIl9bxrcJKqtzvN9eVG-3-MlOfCWwGyEEBGpOkpxX6-x7owRRQIUIwGo89EbwwLCRb-7ipjCH_f9hYXOwuPLaUg6bw3KSUkDdBsRO1xCcWLSrQa4jIQuiQV2Tfo6GyaDjsK4rHKuMLX93CQDtXItb5S2iT42BMC0YYD_KpSKuGdROvnj9wrBvl8Az4R0doDRwB8BTSR1_L3H0Gaox00WpzhFbnUKLs-a0wuFLX0CPo3bUCrcQRXWa6RybifqYf9fQVhJ9cHE_KDPesZEad5x-DocR9s6-NPJGLMQ4eaW0uBEW8XmGQllUcKUILdAbT_j7v4vQUNh7JHW2ih_e0RnH6l6u1dOESQz9YtpLv-Z22ijjmoBencE6hpZIuA0r8sHLHcaeb_vA2imLN0_7mo9sq3MKeYTo2GdBZrfJv9b-meHlQL32OY7TTTpzplYdbr4iQUkPLVdFimi7Y2dCMpQbFI2hQYy32cvd2RMjqsdwfFTmf03lKTd4MdfT35VLQ4Wbpmt-veWLfwb2cSEKeJyKP2-gC4cP_UmFGt7w4aFNkD-rLyYOIdxhkYR6uootEzU9Cy
{'html_attributions': [], 'results': [{'geometry': {'location': {'lat': 47.65769209999999, 'lng': -122.1924837}, 'viewport': {'northeast': {'lat': 47.65979674999999, 'lng': -122.1843905}, 'southwest': {'lat': 47.65407335, 'lng': -122.1998117}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png', 'id': '977d947a2e9

<Response [200]>

In [146]:
a[19]

{'name': 'The Triple Door',
 'geometry': {'lat': 47.608704, 'lng': -122.3374196},
 'rating': 4.5}

In [ ]:
#https://developers.google.com/places/web-service/supported_types
#types supported by google places
#next_page_token

In [210]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params={
    
    #'location':'47.6918452,-122.2226413',
    #'radius':'5000',
    #'type':'bar'
    'pagetoken':'CqQCGgEAAJVtKqN_2D8dUG7koF8KD8AA2RoqaTRRtESH9EJk359BUsO0O0Dv7LCMzNVc7Q1ATutfTOfeTo9PwICP2Y-qcstscN_kFO3yNG_Rnl4mvbJcv7XGr3K4_ZuFq70cxY5s1zLRGf9fBbU4-C2vLdarm0qL3F0IHbd1O5SJ3Zm72413yskvyYbtrw_wAT7zgtOx95ELn6qdvRrR14fuADtO9tZcoTPAy6MLWDGGdaAphDUcgBvPuXJeiEjLwLtN8LFe5v36zLM3pgWHnn1fhCSQQkclWRixoRss8VjdOYNFFqwQH1Im_i_LuT_DUX9uSaSXf9eo44KHm8vh40iDVA4w2GKBGcOvTiceIpDdxriLFpSNsgi6P4E-pATrBoDPcTIxEBIQ353VhfcqSIx7NzuvvWXngRoUv0aG8qsB1EhtFzrgGShL6YKmsVw'
    ,'key':API_KEY
}
res = requests.get(url, params=params)
x = res.json() 
#print(x)
# now x contains list of nested dictionaries 
# we know dictionary contain key value pair 
# store the value of result key in variable y 
y = x['results'] 
# keep looping upto lenght of y 
for i in range(len(y)): 

# Print value corresponding to the 
# 'name' key at the ith index of y 
    print(y[i]['name'],y[i]['geometry']['location'])



Maison DeLille Wine Lounge {'lat': 47.6759539, 'lng': -122.2069466}
The Melody Lynne Vineyard {'lat': 47.6756471, 'lng': -122.2061802}
Azteca Mexican Restaurants {'lat': 47.71100409999999, 'lng': -122.1876683}
Thumra Thai {'lat': 47.7125528, 'lng': -122.1875046}
Lucky 7 Bar and Grill {'lat': 47.7099941, 'lng': -122.1709245}
The Grape Choice {'lat': 47.6757578, 'lng': -122.2070561}
Vovina {'lat': 47.675994, 'lng': -122.207168}
Emerald Garden Chinese Restaurant & Lounge {'lat': 47.70670800000001, 'lng': -122.2097621}
The Game Neighborhood Grill & Bar {'lat': 47.7216077, 'lng': -122.2071704}
jb bungalow {'lat': 47.7052816, 'lng': -122.212214}
Cock & Bull Public House {'lat': 47.7066144, 'lng': -122.2097439}
Flatstick Pub - Kirkland {'lat': 47.67593429999999, 'lng': -122.2071522}
Park Lane Public House {'lat': 47.6759622, 'lng': -122.206039}
Bottle and Bull {'lat': 47.6751436, 'lng': -122.206563}
Papi's Cantina {'lat': 47.6745871, 'lng': -122.2058805}
Eastside Hookah Lounge {'lat': 47.7032

In [188]:
x

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

True

## Meetup api

In [91]:
urlBase='https://api.meetup.com'

In [92]:
urlFindEvent='/find/upcoming_events'

In [96]:
params={
    'lat':'47.6918452',
    'lon':'-122.2226413'
}

In [100]:
res = requests.get('urlBase+urlFindEvent, params=params')
res.json()

{'errors': [{'code': 'authentication_error',
   'message': 'Authenticated member required'}]}

In [29]:
# Python program to get a set of 
# places according to your search 
# query using Google Places API 

# importing required modules 
import requests, json 

# enter your api key here 
api_key = API_KEY

# url variable store url 
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

# The text string on which to search 
query = input('Search query: ') 

# get method of requests module 
# return response object 
r = requests.get(url + 'query=' + query +
						'&key=' + api_key) 

# json method of response object convert 
# json format data into python format data 
x = r.json() 

# now x contains list of nested dictionaries 
# we know dictionary contain key value pair 
# store the value of result key in variable y 
y = x['results'] 

# keep looping upto lenght of y 
for i in range(len(y)): 
	
	# Print value corresponding to the 
	# 'name' key at the ith index of y 
	print(y[i]['name']) 


Search query: Bar
Restaurante La Braña
Casa Mariano
Bodega La Andaluza
Bar Oliva
José's Bar
bar bonsai
Fanáticos del sabor bar restaurante
Bar Susi. La Esquina de las Raciones
Bar RUBÍ
AROMAS PERUANOS BAR RESTAURANTE
Vermut De Grifo Cafe-Bar Muinelo
Bar El Rincon De Bbto
D'Guille Bar
Bar las Cubas
Bar cafeteria El Jamonal
A Tope Bar
Bar Chipiron
Bar restaurante Los Kali 2 S.l.
Bar Marbi
Bar Carrion


{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account',
 'html_attributions': [],
 'results': [],
 'status': 'OVER_QUERY_LIMIT'}

SyntaxError: invalid syntax (<ipython-input-46-3263d53210f2>, line 1)